In [1]:
import pandas as pd
import numpy as np

In [2]:
df_train = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/train.csv')
df_test = pd.read_csv('/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/test.csv')

FileNotFoundError: [Errno 2] No such file or directory: '/media/yanncauchepin/ExternalDisk/Datasets/ComputerVisionImages/digit_recognizer/train.csv'

In [ ]:
distinct_labels = df_train['label'].value_counts()
print(pd.DataFrame(distinct_labels))
classes = len(df_train["label"].unique())

       count
label       
1       4684
7       4401
3       4351
9       4188
2       4177
6       4137
0       4132
4       4072
8       4063
5       3795


In [ ]:
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix

def evaluate_classifier(y_true, y_pred):
    f1 = f1_score(y_true, y_pred, average='weighted')
    precision = precision_score(y_true, y_pred, average='weighted')
    recall = recall_score(y_true, y_pred, average='weighted')
    cm = confusion_matrix(y_true, y_pred)
    
    metrics_df = pd.DataFrame({
        'Metric': ['F1 Score', 'Precision', 'Recall'],
        'Value': [f1, precision, recall]
    })
    
    cm_df = pd.DataFrame(cm, columns=df_train["label"].unique(), index=df_train["label"].unique())
    
    return metrics_df, cm_df

In [ ]:
y_train = np.array(df_train['label'])
X_train = np.array(df_train.drop('label', axis=1))
X_test = np.array(df_test)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.2, stratify=y_train, random_state=42)

In [8]:
X_train_norm = X_train / 255.0
X_valid_norm = X_valid / 255.0
X_test_norm = X_test / 255.0

In [36]:
from sklearn.neighbors import KNeighborsClassifier

knn_classifier = KNeighborsClassifier(n_neighbors=classes)
knn_classifier.fit(X_train_norm, y_train)

y_pred = knn_classifier.predict(X_valid_norm)

knn_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(knn_classifier_assessement)

(      Metric     Value
0   F1 Score  0.962301
1  Precision  0.963080
2     Recall  0.962381,      1    0    4    7    3    5    8    9    2    6
1  817    0    1    0    0    4    4    0    0    1
0    0  934    1    0    0    0    0    2    0    0
4    9   17  789    3    1    2    0   13    0    1
7    2    6    5  831    0   14    0    5    3    4
3    0   10    0    0  783    0    1    1    0   19
5    4    5    0   10    0  720    7    0    3   10
8    5    2    0    0    1    4  815    0    0    0
9    0   10    0    0    1    0    0  862    0    7
2    9   15    2   15    3   18    3    3  732   13
6    4    1    1    3   10    3    0   15    0  801)


In [38]:
from xgboost import XGBClassifier

xgb_classifier = XGBClassifier(n_estimators=1000, max_depth=4, learning_rate=0.1,  eval_metric='logloss')
xgb_classifier.fit(X_train_norm, y_train, verbose=1)

y_pred = xgb_classifier.predict(X_valid_norm)

xgb_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(xgb_classifier_assessement)

(      Metric     Value
0   F1 Score  0.975350
1  Precision  0.975385
2     Recall  0.975357,      1    0    4    7    3    5    8    9    2    6
1  812    0    1    0    2    1    6    0    2    3
0    0  930    2    3    0    0    0    0    2    0
4    0    0  815    4    5    0    0    8    2    1
7    3    2    7  832    3   10    0    3    3    7
3    1    4    0    0  794    0    2    1    1   11
5    4    1    1    7    1  732    6    0    4    3
8    0    1    1    0    0    5  817    0    3    0
9    0    3    4    2    2    1    0  861    1    6
2    3    1    0    2    0    6    3    1  788    9
6    3    2    0    3    4    2    0    9    3  812)


In [39]:
from catboost import CatBoostClassifier

catboost_classifier = CatBoostClassifier(iterations=1000, depth=4, learning_rate=0.1, verbose=2)
catboost_classifier.fit(X_train_norm, y_train)

y_pred = catboost_classifier.predict(X_valid_norm)

catboost_classifier_assessement = evaluate_classifier(y_valid, y_pred)
print(catboost_classifier_assessement)

0:	learn: 2.0962791	total: 397ms	remaining: 6m 36s
2:	learn: 1.8310850	total: 907ms	remaining: 5m 1s
4:	learn: 1.6302713	total: 1.45s	remaining: 4m 48s
6:	learn: 1.4751555	total: 1.96s	remaining: 4m 37s
8:	learn: 1.3460221	total: 2.44s	remaining: 4m 28s
10:	learn: 1.2434894	total: 2.94s	remaining: 4m 24s
12:	learn: 1.1528075	total: 3.46s	remaining: 4m 22s
14:	learn: 1.0777297	total: 3.91s	remaining: 4m 17s
16:	learn: 1.0118651	total: 4.4s	remaining: 4m 14s
18:	learn: 0.9546359	total: 4.84s	remaining: 4m 9s
20:	learn: 0.8999357	total: 5.3s	remaining: 4m 7s
22:	learn: 0.8518069	total: 5.8s	remaining: 4m 6s
24:	learn: 0.8097365	total: 6.33s	remaining: 4m 6s
26:	learn: 0.7709663	total: 6.89s	remaining: 4m 8s
28:	learn: 0.7421569	total: 7.36s	remaining: 4m 6s
30:	learn: 0.7083525	total: 7.89s	remaining: 4m 6s
32:	learn: 0.6796631	total: 8.36s	remaining: 4m 5s
34:	learn: 0.6590664	total: 8.73s	remaining: 4m
36:	learn: 0.6324410	total: 9.23s	remaining: 4m
38:	learn: 0.6080263	total: 9.72s	rem

In [48]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping

mlp_classifier = Sequential([
    Dense(128, activation='relu', input_shape=(X_train_norm.shape[1],)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(10, activation='softmax')
])

mlp_classifier.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
early_stopping = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)
mlp_classifier.fit(
    X_train_norm, 
    y_train, 
    epochs=20, 
    batch_size=64,
    validation_data=(X_valid_norm, y_valid),
    callbacks=[early_stopping]
    )

y_pred_proba = mlp_classifier.predict(X_valid_norm)
y_pred = np.argmax(y_pred_proba, axis=1)

mlp_classifier_assessment = evaluate_classifier(y_valid, np.round(y_pred))
print(mlp_classifier_assessment)

/home/yanncauchepin/Git/.venv/lib/python3.12/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/20


2024-11-01 20:21:25.092408: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 105369600 exceeds 10% of free system memory.


525/525 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.7472 - loss: 0.8038 - val_accuracy: 0.9400 - val_loss: 0.2079
Epoch 2/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9292 - loss: 0.2314 - val_accuracy: 0.9555 - val_loss: 0.1463
Epoch 3/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9518 - loss: 0.1625 - val_accuracy: 0.9620 - val_loss: 0.1266
Epoch 4/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9590 - loss: 0.1356 - val_accuracy: 0.9646 - val_loss: 0.1170
Epoch 5/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9650 - loss: 0.1113 - val_accuracy: 0.9689 - val_loss: 0.1046
Epoch 6/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9714 - loss: 0.0942 - val_accuracy: 0.9692 - val_loss: 0.1033
Epoch 7/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9741 - loss: 0.0840 - val_accuracy: 0.9706 - val_loss: 0.1025
Epoch 8/20
525/525 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.9759 - loss: 0.0752 - val_accuracy: 0.9714 - val_

In [55]:
# !pip install torchvision

  Using cached torch-2.5.1-cp312-cp312-manylinux1_x86_64.whl.metadata (28 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 1.1 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 906.4/906.4 MB 776.0 kB/s eta 0:00:000:0100:21m
  Attempting uninstall: torch
    Found existing installation: torch 2.5.0
    Uninstalling torch-2.5.0:
      Successfully uninstalled torch-2.5.0


In [ ]:
import torch
from torchvision import transforms
from torch.utils.data import DataLoader, TensorDataset

transform = transforms.Compose([
    transforms.Lambda(lambda x: torch.tensor(x.reshape(28, 28), dtype=torch.float32).unsqueeze(0)),  # Reshape and add channel
    transforms.Resize((224, 224)),                                                                  # Resize to 224x224
    transforms.Lambda(lambda x: x.expand(3, -1, -1)),                                               # Duplicate channels to create RGB
    transforms.Normalize((0.5,), (0.5,))                                                            # Normalize to [-1, 1]
])

# Apply transformations on training and validation sets
X_train_transformed = torch.stack([transform(x) for x in X_train]) 
X_valid_transformed = torch.stack([transform(x) for x in X_valid])  # Transform each item in X_valid

# Convert labels to tensors
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
y_valid_tensor = torch.tensor(y_valid, dtype=torch.long)

# Create TensorDatasets and DataLoaders for batch processing
train_dataset = TensorDataset(X_train_transformed, y_train_tensor)
valid_dataset = TensorDataset(X_valid_transformed, y_valid_tensor)
train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=2)

In [ ]:
from transformers import ViTForImageClassification

vit_classifier = ViTForImageClassification.from_pretrained(
    'google/vit-base-patch16-224-in21k', 
    num_labels=10 
)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
vit_classifier.to(device)

In [ ]:
from transformers import AdamW
from tqdm import tqdm

optimizer = AdamW(vit_classifier.parameters(), lr=1e-5)

epochs = 3
for epoch in range(epochs):
    vit_classifier.train()
    train_loss = 0
    for batch in tqdm(train_loader):
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        outputs = vit_classifier(inputs)
        loss = outputs.loss
        train_loss += loss.item()
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
    
    avg_train_loss = train_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{epochs}, Training Loss: {avg_train_loss:.4f}")

In [ ]:
from sklearn.metrics import accuracy_score, classification_report

vit_classifier.eval()
all_vit_classifier_preds, all_vit_classifier_labels = [], []

with torch.no_grad():
    for batch in valid_loader:
        inputs, labels = batch
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = vit_classifier(inputs)
        preds = torch.argmax(outputs.logits, axis=1)
        
        all_vit_classifier_preds.extend(preds.cpu().numpy())
        all_vit_classifier_labels.extend(labels.cpu().numpy())


accuracy = accuracy_score(all_vit_classifier_labels, all_vit_classifier_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(all_vit_classifier_labels, all_vit_classifier_preds))


In [ ]:
vit_classifier_assessement = evaluate_classifier(all_vit_classifier_labels, all_vit_classifier_preds)
print(vit_classifier_assessement)

In [ ]:
import torch.nn as nn
import torch.optim as optim
from sklearn.metrics import accuracy_score, classification_report

class DigitCNN(nn.Module):
    def __init__(self):
        super(DigitCNN, self).__init__()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)
        self.fc1 = nn.Linear(64 * 7 * 7, 128)
        self.fc2 = nn.Linear(128, 10)
        self.dropout = nn.Dropout(0.25)
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))
        x = self.pool(torch.relu(self.conv2(x)))
        x = x.view(-1, 64 * 7 * 7)  # Flatten for fully connected layer
        x = torch.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x

transform = transforms.Compose([
    transforms.Lambda(lambda x: x.reshape(28, 28).astype(np.float32)),
    transforms.ToTensor(), 
])

X_train_transformed = torch.stack([transform(x) for x in X_train])
X_valid_transformed = torch.stack([transform(x) for x in X_valid])
y_train_tensor = torch.tensor(y_train)
y_valid_tensor = torch.tensor(y_valid)

train_dataset = TensorDataset(X_train_transformed, y_train_tensor)
valid_dataset = TensorDataset(X_valid_transformed, y_valid_tensor)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=8)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
cnn_classifier = DigitCNN().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(cnn_classifier.parameters(), lr=0.001)

num_epochs = 10
for epoch in range(num_epochs):
    cnn_classifier.train()
    running_loss = 0.0
    for inputs, labels in train_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        optimizer.zero_grad()
        outputs = cnn_classifier(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
    
    avg_loss = running_loss / len(train_loader)
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {avg_loss:.4f}")

cnn_classifier.eval()
all_cnn_classifier_preds, all_cnn_classifier_labels = [], []

with torch.no_grad():
    for inputs, labels in valid_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        outputs = cnn_classifier(inputs)
        preds = torch.argmax(outputs, axis=1)
        
        all_cnn_classifier_preds.extend(preds.cpu().numpy())
        all_cnn_classifier_labels.extend(labels.cpu().numpy())

# Calculate accuracy and classification report
accuracy = accuracy_score(all_cnn_classifier_labels, all_cnn_classifier_preds)
print(f"Validation Accuracy: {accuracy:.4f}")
print("Classification Report:\n", classification_report(all_cnn_classifier_labels, all_cnn_classifier_preds))
cnn_classifier_assessement = evaluate_classifier(all_cnn_classifier_labels, all_cnn_classifier_preds)
print(cnn_classifier_assessement)